
grafana llm service?


In [ ]:
const {Remarkable} = require('remarkable')
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true})

const express = require('express')
const cors = require('cors')


let initialChatHistory
let session
async function handleRequest(req, res) {
  if(!session) {
    const {createSession} = await importer.import('llama vision')
    session = await createSession(process.env.DEFAULT_MODEL || 'gemma-3-27b-it-Q6_K.gguf', req.body.messages[0].content)
    initialChatHistory = session.getChatHistory();
  }

  if(req.body.messages && req.body.messages.length) {
    req.body.message = req.body.messages[req.body.messages.length-1].content
  }

  console.log('User: ' + (req.body.content || req.body.message))
  let result = await session.prompt(req.body.content || req.body.message, {
    //temperature: temperature,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })

  console.log('AI: ' + result)
  session.setChatHistory(initialChatHistory)

  let code = result.matchAll(/```(json|bash|javascript|code|markdown)*\n[\s\S]*?\n*```/gi)
  if(code) {
    result = result.replaceAll(/^```(json|bash|javascript|code|markdown)\n*|\n*```\s*$/gi, '')
  }


  if(req.headers.accept == 'application/json') {
    return res.json({ content: result, object: result });
  }
  const mdHtml = md.render(result)
  return res.send(mdHtml)
}

async function startServer() {
  var app = express()
  app.use(cors({
    credentials: true,
    origin: '*' 
  }))
  app.use(express.json({ limit: '20mb' }))
  app.post('/', handleRequest)
  app.post('/v1/chat/completions', handleRequest)

  let port = process.env.CHAT_PORT || 9393
  app.listen(port, () => console.log('Listening on http://localhost:' + port))
}

module.exports = startServer
